## SGD + Grid(Random) Search 방식으로 Ridge Regression 구현하기
### - Grid(Random) Search = hyperparameter 탐색 + cross validation
#### [10. 교차 검증과 그리드 서치]에서는 Grid(Random) Search방식으로 Decision Tree(Classification)를 구현했다.
#### Grid(Random) Search에 SGD방식을 추가하여 Ridge Regression/Lasso Regression(Regression)을 구현해보자.

In [ ]:
import pandas as pd

# 농어 56마리 [길이, 높이, 두께] 데이터 받아오기
df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy() # 넘파이 배열로 변환
print(perch_full)

[[ 8.4   2.11  1.41]
 [13.7   3.53  2.  ]
 [15.    3.82  2.43]
 [16.2   4.59  2.63]
 [17.4   4.59  2.94]
 [18.    5.22  3.32]
 [18.7   5.2   3.12]
 [19.    5.64  3.05]
 [19.6   5.14  3.04]
 [20.    5.08  2.77]
 [21.    5.69  3.56]
 [21.    5.92  3.31]
 [21.    5.69  3.67]
 [21.3   6.38  3.53]
 [22.    6.11  3.41]
 [22.    5.64  3.52]
 [22.    6.11  3.52]
 [22.    5.88  3.52]
 [22.    5.52  4.  ]
 [22.5   5.86  3.62]
 [22.5   6.79  3.62]
 [22.7   5.95  3.63]
 [23.    5.22  3.63]
 [23.5   6.28  3.72]
 [24.    7.29  3.72]
 [24.    6.38  3.82]
 [24.6   6.73  4.17]
 [25.    6.44  3.68]
 [25.6   6.56  4.24]
 [26.5   7.17  4.14]
 [27.3   8.32  5.14]
 [27.5   7.17  4.34]
 [27.5   7.05  4.34]
 [27.5   7.28  4.57]
 [28.    7.82  4.2 ]
 [28.7   7.59  4.64]
 [30.    7.62  4.77]
 [32.8  10.03  6.02]
 [34.5  10.26  6.39]
 [35.   11.49  7.8 ]
 [36.5  10.88  6.86]
 [36.   10.61  6.74]
 [37.   10.84  6.26]
 [37.   10.57  6.37]
 [39.   11.14  7.49]
 [39.   11.14  6.  ]
 [39.   12.43  7.35]
 [40.   11.93

In [ ]:
import numpy as np

# 농어 56마리 무게 데이터
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0,
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0,
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0,
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0,
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0,
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0,
     1000.0, 1000.0]
     )

In [ ]:
# train set과 test set으로 나누기 (기본 train set: 80%, test set: 20%)
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

In [ ]:
# 새로운 특성 만들기 (기본 degree값은 2)
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# 객체 생성 # 5제곱까지 특성 만들기 (기본 degree값은 2)
poly = PolynomialFeatures(degree=5, include_bias=False)

# 객체 training
poly.fit(train_input)
# train_input 변환
train_poly = poly.transform(train_input)
# test_input 변환
test_poly = poly.transform(test_input)

In [ ]:
print(train_poly.shape)

(42, 55)


In [ ]:
# Z-Score Normalization
from sklearn.preprocessing import StandardScaler

# 객체 생성
ss = StandardScaler()
# 객체 훈련 # train input data에 대해 mean값과 std값이 구해짐
ss.fit(train_poly)

# train_poly 변환(정규화)
train_scaled = ss.transform(train_poly)
# test_poly 변환(정규화)
test_scaled = ss.transform(test_poly) # 반드시 train set의 통계 값으로 변환해야함(반드시 train set으로 fit)

### 모델 라이브러리 불러오기

In [ ]:
# 확률적 경사 하강법(Regression) 사용(loss : 손실 함수, max_iter : 에포크 수, tol : 성능향상 최솟값(성능향상되지 않으면 훈련 종료))
from sklearn.linear_model import SGDRegressor

# 주어진 범위에서 고르게 값을 뽑음(균등 분포에서 샘플링) # uniform은 실숫값 # randint는 정숫값
from scipy.stats import uniform, randint

# RandomSearch 사용 (기본 k값은 5)
from sklearn.model_selection import RandomizedSearchCV

### Ridge Regression(L2 규제)
#### 하이퍼파라미터
#### - alpha는 0.0001부터 100사이의 값을 탐색
#### - max_iter는 10부터 300사이의 값을 탐색
#### 샘플링 횟수는 150으로 고정

In [ ]:
# 모델 생성
sr = SGDRegressor(loss='squared_error', penalty='l2', random_state=42)

# 탐색할 하이퍼파라미터를 딕셔너리로 생성
params = ?

# 객체 생성(모델 생성)
gs = RandomizedSearchCV(sr, params, ?, n_jobs=-1, random_state=42) # n_iter : 샘플링 횟수 # n_jobs : 사용할 CPU 코어 수, -1일 경우 시스템에 있는 모든 CPU 코어 사용

# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_scaled, train_target)

RandomizedSearchCV(estimator=SGDRegressor(random_state=42), n_iter=150,
                   n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c4ea4662dd0>,
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c4e9c23d2a0>},
                   random_state=42)

In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)

{'alpha': 1.3265961159866528, 'max_iter': 98}


In [ ]:
# 최적의 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(np.max(gs.cv_results_['mean_test_score']))

0.9829499467637485


In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터로 전체 train set를 훈련시킨 모델 불러오기
dt = gs.best_estimator_

# 모델 평가
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

0.9853559555787426
0.9658535133835567


In [ ]:
# 최적의 하이퍼파라미터로 다시 모델 만들기
# 모델 생성
sr = SGDRegressor(loss='squared_error', penalty='l2', alpha=1.3265961159866528, max_iter=98, tol=None, random_state=42)
# 모델 훈련
sr.fit(train_scaled, train_target)

# 모델 평가
print(sr.score(train_scaled, train_target))
print(sr.score(test_scaled, test_target))

0.9866177252342649
0.9619777105051306


### Lasso Regression(L1 규제)
#### 하이퍼파라미터
#### - alpha는 0.0001부터 100사이의 값을 탐색
#### - max_iter는 10부터 300사이의 값을 탐색
#### 샘플링 횟수는 150으로 고정

In [ ]:
# 모델 생성
sr = SGDRegressor(loss='squared_error', penalty='l1', random_state=42)

# 탐색할 하이퍼파라미터를 딕셔너리로 생성
params = ?

# 객체 생성(모델 생성)
gs = RandomizedSearchCV(sr, params, ?, n_jobs=-1, random_state=42) # n_iter : 샘플링 횟수 # n_jobs : 사용할 CPU 코어 수, -1일 경우 시스템에 있는 모든 CPU 코어 사용

# 모델 훈련 # params에 들어있는 하이퍼파라미터의 값을 바꿔가며 훈련하고, 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 모델을 훈련
gs.fit(train_scaled, train_target)

RandomizedSearchCV(estimator=SGDRegressor(penalty='l1', random_state=42),
                   n_iter=150, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c4e9c23cb20>,
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c4e9c297e20>},
                   random_state=42)

In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터
print(gs.best_params_)

{'alpha': 3.1430185686734253, 'max_iter': 240}


In [ ]:
# 최적의 매개변수에서 수행한 교차 검증의 최종 점수(평균 점수)
print(np.max(gs.cv_results_['mean_test_score']))

0.9766034329195229


In [ ]:
# 랜덤 서치로 찾은 최적의 하이퍼파라미터로 전체 train set를 훈련시킨 모델 불러오기
dt = gs.best_estimator_

# 모델 평가
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

0.9856583886292948
0.9676802550388461


In [ ]:
# 최적의 하이퍼파라미터로 다시 모델 만들기
# 모델 생성
sr = SGDRegressor(loss='squared_error', penalty='l1', alpha=3.1430185686734253, max_iter=240, tol=None, random_state=42)
# 모델 훈련
sr.fit(train_scaled, train_target)

# 모델 평가
print(sr.score(train_scaled, train_target))
print(sr.score(test_scaled, test_target))

0.9888708856843276
0.9799359896497981
